In [ ]:
import random
import torch
from yorzoi.dataset import GenomicDataset
from yorzoi.model.borzoi import Borzoi

model = Borzoi.from_pretrained("tom-ellis-lab/yorzoi")
model.to("cuda:0")
model.eval()

def random_dna_sequence(length):
    return ''.join(random.choices('ACGT', k=length))

sequences = torch.stack([torch.tensor(GenomicDataset.one_hot_encode((random_dna_sequence(4992))), dtype=torch.float32) for _ in range(5)])

print(f"\nPredicting RNA-seq coverage for {sequences.shape[0]} sequences\n")

sequences = sequences.to("cuda:0")

with torch.autocast(device_type="cuda"):
    predictions = model(sequences)

print(f"Made predictions for {predictions.shape[0]} sequences, {predictions.shape[1]} tracks each (sense: {predictions.shape[1]//2}, antisense: {predictions.shape[1]//2}), \nacross {predictions.shape[2]} bins (resolution=10 => 3000 bp). The first half of the tracks are \nthe sense strand, the second half are the antisense strand. The first track \nof the first half belongs to the same RNA-seq experiment as the first track \nof the second half and so on.")